In [ ]:
from mnist_dataset import ImageAndLabelSet
from networks import FCNet, ResNet6
import numpy as np
from os import path
import matplotlib.pyplot as plt

In [ ]:
# Get the dataset
mnist_data_root = path.expanduser("~/data/mnist")

image_and_label_set = ImageAndLabelSet(path.join(mnist_data_root, 'train-images-idx3-ubyte'),
                                       path.join(mnist_data_root, 'train-labels-idx1-ubyte'),
                                       training_fraction=(5.0/6.0))

In [ ]:
image_and_label_set.get_validation_batches()

In [ ]:
?Generator

In [ ]:
from typing import Generator

def foo(x: Generator[int, None, None]):
    return None

# Fully Connected Network

In [ ]:
# Define a network that downsamples and then trains a single FC layer
fc_net = FCNet()

for b in range(1000):
    image_batch, label_batch = image_and_label_set.get_next_training_batch(batch_size=32)
    image_batch = np.reshape(image_batch, (-1, 28, 28, 1)) # Add a channel dimension
    
    fc_net.train_batch(image_batch, label_batch)
    
    if b % 100 == 0:
        print('batch {}: CE: {}\tacc: {}'.format(b, fc_net._running_ce.get(), fc_net._running_acc.get()))

In [ ]:
# Plot the training results for this network
fig, ax = plt.subplots()
ax.plot(fc_net._ce_history)
ax2 = ax.twinx()
ax2.plot([100.0*(1.0 - x) for x in fc_net._acc_history], 'r')
ax2.set_yscale('log')
ax2.set_ylabel('Error rate (%)')
plt.show()

# ResNet6

In [ ]:
# Define a network that downsamples and then trains a single FC layer
res_net = ResNet6()

for b in range(10000):
    image_batch, label_batch = image_and_label_set.get_next_training_batch(batch_size=32)
    image_batch = np.reshape(image_batch, (-1, 28, 28, 1)) # Add a channel dimension
    
    res_net.train_batch(image_batch, label_batch)
    
    if b % 500 == 0:
        print('batch {}: CE: {}\tacc: {}'.format(b, res_net._running_ce.get(), res_net._running_acc.get()))
        
    if b % 500 == 0:
        res_net.evaluate_on_batch_generator(image_and_label_set.get_validation_batches())
            
        

In [ ]:
res_net.show_report()